In [ ]:
import os
import numpy as np 
import pandas as pd

import shutil
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *

from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

path= '../input/rethink-us/Rethink_UX'
k= '../input/rethink-us/Rethink_UX/train.csv'
p ='../input/rethink-us/Rethink_UX/test.csv'

In [ ]:
train=pd.read_csv(k)
test=pd.read_csv(p)
train.head(5),test.head(5)

In [ ]:
np.random.seed(1786)
ROOT = "/tmp/data"

In [ ]:
def read_data(root):
    train_df = pd.read_csv(k)
    test_df = pd.read_csv(p)
    return train_df, test_df

In [ ]:
class SaveBestModel(Recorder):
    def __init__(self, learn,name='best_model'):
        super().__init__(learn)
        self.name = name
        self.best_loss = None
        self.best_acc = None
        self.save_method = self.save_when_acc
        
    def save_when_acc(self, metrics):        
        loss, acc = metrics[0], metrics[1]
        if (self.best_acc is None) or (acc > self.best_acc) or (loss < self.best_loss):
            self.best_acc = acc
            self.best_loss = loss
            self.learn.save(f'{self.name}')
            print("Save the best acc {:.5f}".format(self.best_acc))
        elif acc == self.best_acc and  loss < self.best_loss:
            self.best_loss = loss
            self.learn.save(f'{self.name}')
            print("Acc is eq,Save the lower loss {:.5f}".format(self.best_loss))
            
    def on_epoch_end(self,last_metrics=MetricsList,**kwargs:Any):
        self.save_method(last_metrics)

In [ ]:
#Path(ROOT).mkdir(exist_ok=True, parents=True)
src1 = "../input/rethink-us/Rethink_UX"
shutil.copytree(src1, ROOT)

In [ ]:
if __name__=="__main__":
    train_df, test_df = read_data(ROOT)
    print(train_df.shape, test_df.shape)
    
    cvlist = list(StratifiedKFold(16, random_state=12345786).split(train_df, train_df.category))
    tfms1 = get_transforms(do_flip=True, flip_vert=True, max_rotate=.0, max_zoom=2.0)
    #tfms1 = get_transforms(flip_vert=False, max_zoom=1.5, max_warp=0, do_flip=False, xtra_tfms=[cutout()])
    #tfms1 = get_transforms(max_zoom=2.0)
    test_preds_all = []
    val_preds_all = []
    for i in range(10):
        print("Starting fold {}".format(i))
        tr_idx, val_idx = cvlist[i]
        
        print(tr_idx.shape, val_idx.shape)
        src = (ImageList.from_df(train_df, path=ROOT).split_by_idxs(tr_idx, val_idx)
                                                            .label_from_df())
        data = ImageDataBunch.create_from_ll(src, ds_tfms=tfms1, size=256, bs=32, resize_method=3).normalize(imagenet_stats)
        data.add_test(ImageList.from_df(test_df, path=ROOT))
        learn = cnn_learner(data, models.densenet201 , metrics=accuracy, ps=0.5)
        cb = SaveBestModel(learn, name="bestmodel_{}".format(i))
        learn.fit_one_cycle(4)
        learn.unfreeze()
        learn.fit_one_cycle(10, max_lr=1e-4, callbacks=[cb])
        learn.fit_one_cycle(10, max_lr=5e-5, callbacks=[cb])
        learn.fit_one_cycle(10, max_lr=1e-5, callbacks=[cb])
        learn.fit_one_cycle(10, max_lr=5e-6, callbacks=[cb])
        learn.fit_one_cycle(4, max_lr=1e-6, callbacks=[cb])
        learn.load("bestmodel_{}".format(i))
        val_preds, y = learn.TTA(ds_type=DatasetType.Valid)
        val_preds = np.exp(val_preds.numpy())
        print("F1 score for this fold ",f1_score(y.numpy(), np.argmax(val_preds,axis=1), average='weighted'))
        test_preds = np.exp(learn.TTA(ds_type=DatasetType.Test)[0].numpy())
        test_preds_all.append(test_preds)
        val_preds_all.append(val_preds)
        fname = "bestmodel_{}.pth".format(i)
        src = str(Path(ROOT) / "models" / fname)
        shutil.copy(src, fname)
    test_preds_all = np.mean(test_preds_all, axis=0)
    val_preds_all = np.concatenate(val_preds_all, axis=0)

    np.save("test_preds.npy", test_preds_all)
    np.save("val_preds.npy", val_preds_all)

In [ ]:
l = len(test_df.path)
img = []
for i in range(l):
    path = test_df.path[i]
    img.append(os.path.split(path)[1])

df = pd.DataFrame(img,columns=['Filename'])
df["Labels"] = np.argmax(test_preds, axis=1)

In [ ]:
y=[]
for i in df['Labels']:
    y.append(val[i])
df['Labels']=y

df.head(2)

In [ ]:
df.to_csv("submission.csv", index=False)